## What's an HTTP API?

API stands for application programming interface. It's basically set of (hopefully) convenient methods for interacting with and controlling a program. HTTP APIs make these methods availble via the HTTP protocol. This is the same protocol most of us use daily to browse the web. To use HTTP we send requests to a resource and expect to get a response. For example, when you type https://en.wikipedia.org/wiki/Archive into your web browser, you're sending an HTTP GET request to Wikipedia, and you expect to get the article on Archives back.

We do other types of HTTP requests through our web browsers too, for example, when you fill in a login form with your username and password and hit the login button, you're sending a POST request.: https://en.wikipedia.org/wiki/Hypertext_Transfer_Protocol#Request_methods

## Let's make some HTTP requests in python

You can make HTTP requests from python just like you can from your web browser. To do that, we'll use a set of tools called the requests library. To get the request library, we have to import it like so:

In [ ]:
import requests

In [ ]:
# lets make our first request, The reqeusts library will
# be our browser, and we'll point it at the same Wikipedia article:
requests.get('https://en.wikipedia.org/wiki/Archive')

You should have got something back like `<Response [200]>`. 200 is the HTTP status code for a successful GET request (you're probably familiar with 404 which is the status code for not found). It's not super useful just just have that response spit back at us, so usually when you make a request, you save it to a variable like below. That way you can look at the response's content and other attributes later.

In [ ]:
my_request = requests.get('https://en.wikipedia.org/wiki/Archive')

In [ ]:
# We've saved our request, and now we can do stuff with it
# like print out the body of the response if we want
my_request.content

In [ ]:
# you can also print out the status code, try that below. 
# the syntax is the same, exectp intstead of content, you're
# asking for status_code. You can use the same syntax.

## Log in to the Aspace API 

Now that we're experts with the reqeusts library, let's turn our attention to Archivesspace. The first thing we need to do is log in, which is documented in the Aspace API docs here: https://archivesspace.github.io/archivesspace/api/#authentication. 

We can return to the login request later to understand how it works, but for now, fill in connection information:

In [ ]:
# we need our request making tool, which we 
# can get by importing it like so:
import requests

# first, we'll save bits of connection information (on the right)
# as variables (on the right)

USER = ''
PASS = ''
HOST = ''

# Here's our authentication function. It will return a session
# token if it works, or the value False if if doesn't


def aspace_auth(host, username, password):
    auth = requests.post(host + '/users/' + username + '/login',
                        params={'password' : password})
    if auth.status_code == 200:
        token = auth.json()['session']
        headers = {'X-ArchivesSpace-Session': token}
        return(headers)
    else:
        return(False)

headers = aspace_auth(HOST, USER, PASS)
print(headers)

## GET-ing a record out of Aspace

Now that we're logged in, we're ready to make our first requests in aspace. The first step is to take a look at the Archivesspace API documentation. In particular, let's look at the documentation for how to get a digital object:

https://archivesspace.github.io/archivesspace/api/#get-repositories-repo_id-digital_objects-id

Let's break down the pattern they gave us in the documention:

`/repositories/:repo_id/digital_objects/:id`

A few key points:
- calls are realative to you hostname
- parts with a ":"are variables

So this call is fairly self explanitory, we're asking to get a digital object identified by ":id", which is part of a repository ":repo_id".

Try putting together a call based on a a digital object you can find by looking at the Aspace web interface. As a hint, the Special Collections repo_id is 2, and an objects ID is the last number you see in the URL when you're viewing it.

In [ ]:
# Our first get request to ask for an object. Edit the pattern to select your object:
record_req = requests.get(
    'http://someaspace.somewhere.org:8089/repositories/:repo_id/digital_objects/:id',
    headers=headers
)
    

In [ ]:
# we can check our status code, or request content below to see
# what we got back. I'm calling the .json() method on our
# request result to show the request body as JSON.

record = record_req.json()

# print the record to the console
record

You should see JSON data that looks like it represents the object you found in your browser. 

### Edit a record 

Let's try editing some aspect of the data here, and then sending that change back to the API. We'll change the title of the digital object to test.

In [ ]:
# remember we can select a piece of our JSON data by naming the 
# key in the format object['key']

# we can print out the title field to see what it looks like now
record['title']

In [ ]:
# Looks like the title is a string, so let's replace it with a new string
record['title'] = "SOME NEW TITLE"

# Now we'll print the record to the console again to see if it worked:
record

## Pause and Recap

Questions?

Here's what we've done so far:
- Used the API to download a copy of a record as JSON to our laptops
- Edited our local copy of the record by replacing the title field

To Do:
- Send the changes we've made locally back to Aspace (i.e. edit the remote record)

## POST-ing our record back into Aspace

To complete our circle, we'll write our changes to Aspace. Let's look at the API documentation for sending our record back up: https://archivesspace.github.io/archivesspace/api/#post-repositories-repo_id-digital_objects-id

It should look familiar, in fact the url we'll be sending our request to is the same. The key differences, are that we're using a different HTTP method (POST instead of GET) and we're sending some data along with our request. The syntax for posting data with our Python Requests library is like so:
```python
request_result = requests.post("http://example.com",
                                data="some data",
                                headers=headers_variable)
```
Notice it's more or less the same, except we've added the data= argument and we've chagned get to post.

### Technical note about posting data

Aspace is expecting your data to come as a string, but we've been working with our data as JSON in python, which is a different data type. When we're done editing the data and ready to post it, we'll convert it to a string. Python's json library has a method to do this. They syntax is: `json.dumps(json_data)`.


In [ ]:
# Let's try posting our object back up and break down how the request works:

post_result = requests.post('http://someaspace.somewhere.org:8089/repositories/:repo_id/digital_objects/:id',
                             data=json.dumps(record),
                             headers=headers)

In [ ]:
# We can see what kind of response we got back the same way we did with our get reqeust
post_result.json()

## Wrap up and Questions 

We've made it through a complete simple workflow to edit one item via the API in Aspace. In the followng parts of the workshop, we'll look at ways to operate on multiple records, and more sophistacated ways to retrieve records, but the workflow will remain the same:

1. Get records from Aspace
2. Edit them Locally
3. Post them back up to Aspace